In [1]:
pip install torch torchvision transformers

     |████████████████████████████████| 501kB 6.9MB/s 
     |████████████████████████████████| 870kB 15.9MB/s 
     |████████████████████████████████| 1.0MB 30.1MB/s 
     |████████████████████████████████| 3.7MB 46.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=639deaf8439443df8eafa10620cb3ccdedbf1294fc5a2a3fd8ababc3303af40d
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [2]:
!git clone https://github.com/huggingface/transformers \
&& cd transformers 
# && git checkout a3085020ed0d81d4903c50967687192e3101e770 

Cloning into 'transformers'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 22614 (delta 0), reused 1 (delta 0), pack-reused 22609
Receiving objects: 100% (22614/22614), 13.26 MiB | 14.27 MiB/s, done.
Resolving deltas: 100% (16208/16208), done.


In [3]:
!git clone https://github.com/nyu-mll/GLUE-baselines.git 
!git clone https://github.com/wasiahmad/paraphrase_identification.git \
&& python GLUE-baselines/download_glue_data.py --data_dir glue_data --tasks all --path_to_mrpc=paraphrase_identification/dataset/msr-paraphrase-corpus

Cloning into 'GLUE-baselines'...
remote: Enumerating objects: 886, done.
remote: Total 886 (delta 0), reused 0 (delta 0), pack-reused 886
Receiving objects: 100% (886/886), 1.47 MiB | 3.03 MiB/s, done.
Resolving deltas: 100% (609/609), done.
Cloning into 'paraphrase_identification'...
remote: Enumerating objects: 68, done.
remote: Total 68 (delta 0), reused 0 (delta 0), pack-reused 68
Unpacking objects: 100% (68/68), done.
Checking out files: 100% (45/45), done.
	Completed!
	Completed!
Processing MRPC...
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!


In [4]:
!mkdir dataset \
&& cd dataset \
&& wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json \
&& wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2020-03-24 02:22:12--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   263MB/s    in 0.2s    

2020-03-24 02:22:13 (263 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2020-03-24 02:22:13--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.110.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       1

In [5]:
!ls

dataset		glue_data		   sample_data
GLUE-baselines	paraphrase_identification  transformers


In [6]:
import pandas as pd
import numpy as np
import codecs
import re
from collections import Counter
from matplotlib import pyplot as plt
from operator import itemgetter
from tqdm import tqdm_notebook

import nltk
from nltk.corpus import stopwords
import string
from nltk.stem.porter import PorterStemmer 
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
nltk.download('punkt')

## Torch Modules
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler


import os
import time

from transformers import (
    RobertaConfig, 
    RobertaModel,
    RobertaTokenizer,
    RobertaForSequenceClassification,
    AlbertConfig,
    AlbertForQuestionAnswering,
    AlbertTokenizer,
    squad_convert_examples_to_features
)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
## Check if Cuda is Available
print(torch.cuda.is_available())

True


In [0]:
dataset_url = 'https://raw.githubusercontent.com/DhenPadilla/KSQuAD/master/Bio_GCSE.csv'
df = pd.read_csv(dataset_url)
passage_df = df

In [9]:
passage_df.head()

,Main category,Section,Sub-section,Text,Questions
0,Cells and Organisms,Cell Strucuture and transport,Microscopes,"Living things are all made up of cells, but mo...",Name one advantage and one disadvantage of a l...
1,Cells and Organisms,Cell Strucuture and transport,Animal and plant cells,The cells that make up your body are typical a...,Name the main structures you would expect to f...
2,Cells and Organisms,Cell Strucuture and transport,Eukaryotic and prokaryotic cells,Animal and plant cells are example’s of eukary...,Describe what flagella are ? \nName one use of...
3,Cells and Organisms,Cell Strucuture and transport,Specialisation in animal cells,Although the smallest living organisms are onl...,Name one adaption for the specialised nerver c...
4,Cells and Organisms,Cell Strucuture and transport,Specialisation in plant cells,Animals are not the only organisms to have cel...,Name one adaption for the specialised root hai...


In [0]:
def remove_nulls(dataset, print_out=True):
  dataset.dropna(subset=['question'], inplace=True)
  dataset.reset_index(drop=True, inplace=True)
  for label in dataset["label"].unique():
    if(print_out):
      print("Class: {}, # questions: {}".format(label, dataset.label.value_counts()[label]))
  return dataset

In [44]:
# MAIN CATEGORY DATA SET
category_dataset = pd.DataFrame(columns = ['question', 'label'])

for idx, passage in enumerate(passage_df["Questions"][:]):
  for question in passage.split("\n"):
    small_frame = pd.Series({"question": question, "label": passage_df["Main category"][idx]}) 
    category_dataset = category_dataset.append(small_frame, ignore_index=True)
remove_nulls(category_dataset)

print(category_dataset["label"].size)

Class: Cells and Organisms, # questions: 605
Class: Disease and bioenergetics, # questions: 342
Class: Biological responses, # questions: 80
Class: Genetics and reproduction, # questions: 209
1236


In [71]:
# MAIN CATEGORY -> SECTION DATA SET
section_dataset = pd.DataFrame(columns = ['question', 'label'])

for idx, passage in enumerate(passage_df["Questions"][:]):
  for question in passage.split("\n"):
    small_frame = pd.Series({"question": question, "label": passage_df["Section"][idx]}) 
    section_dataset = section_dataset.append(small_frame, ignore_index=True)
remove_nulls(section_dataset)

print(section_dataset["label"].size)
print(section_dataset["question"][23])

Class: Cell Strucuture and transport, # questions: 305
Class: Cell Division, # questions: 74
Class: Organisation and the digestive system, # questions: 99
Class: Organising Animals and Plants, # questions: 127
Class: Communicable diseases, # questions: 127
Class: Preventing and treating disease, # questions: 69
Class: Non-Communicable diseases, # questions: 76
Class: Photosynthesis, # questions: 28
Class: Respiration, # questions: 42
Class: The human nervous system, # questions: 80
Class: Reproduction, # questions: 76
Class: Genetics and evolution, # questions: 133
1236
What is the resolving power of an electron microscope?


In [0]:
# SECTION -> SUBSECTION DATA SETS (Dictionary)
subsection_datasets = {}

for section in passage_df["Section"].unique():
  dataset = pd.DataFrame(columns = ['question', 'label'])
  section_df = passage_df.loc[passage_df["Section"] == section]
  for idx, row in section_df.iterrows():
    for question in row["Questions"].split("\n"):
      small_frame = pd.Series({"question": question, "label": section_df["Sub-section"][idx]}) 
      dataset = dataset.append(small_frame, ignore_index=True)
      remove_nulls(dataset, print_out=False)
      subsection_datasets[section] = dataset

In [63]:
for section, df in subsection_datasets.items():
  print("Class: {}, # questions: {}".format(section, df["label"].size))

Class: Cell Strucuture and transport, # questions: 305
Class: Cell Division, # questions: 74
Class: Organisation and the digestive system, # questions: 99
Class: Organising Animals and Plants, # questions: 127
Class: Communicable diseases, # questions: 127
Class: Preventing and treating disease, # questions: 69
Class: Non-Communicable diseases, # questions: 76
Class: Photosynthesis, # questions: 28
Class: Respiration, # questions: 42
Class: The human nervous system, # questions: 80
Class: Reproduction, # questions: 76
Class: Genetics and evolution, # questions: 133


In [0]:
def label_to_index(dataset):
  label_to_ix = {}
  for label in dataset.label:
        if label not in label_to_ix:
            label_to_ix[label]=len(label_to_ix)
  return label_to_ix

In [51]:
# Main Category Labels
main_category_labels = label_to_index(category_dataset)
print(main_category_labels)

{'Cells and Organisms': 0, 'Disease and bioenergetics': 1, 'Biological responses': 2, 'Genetics and reproduction': 3}


In [52]:
# Section labels
section_labels = label_to_index(section_dataset)
print(section_labels)

{'Cell Strucuture and transport': 0, 'Cell Division': 1, 'Organisation and the digestive system': 2, 'Organising Animals and Plants': 3, 'Communicable diseases': 4, 'Preventing and treating disease': 5, 'Non-Communicable diseases': 6, 'Photosynthesis': 7, 'Respiration': 8, 'The human nervous system': 9, 'Reproduction': 10, 'Genetics and evolution': 11}


In [64]:
# Sub-Section labels

subsection_labels = {}

for section, df in subsection_datasets.items():
  labels = label_to_index(df)
  subsection_labels[section] = labels
  print(section)
  print(labels)
  print("")

Cell Strucuture and transport
{'Microscopes': 0, 'Animal and plant cells': 1, 'Eukaryotic and prokaryotic cells': 2, 'Specialisation in animal cells': 3, 'Specialisation in plant cells': 4, 'Diffusion': 5, 'Osmosis': 6, 'Osmosis in plants': 7, 'Active transport': 8, 'Exchanging materials': 9}

Cell Division
{'Cell division': 0, 'Growth and differentiation': 1, 'Stem cells': 2, 'Stem cell dilemmas': 3}

Organisation and the digestive system
{'Tissues and organs': 0, 'The human digestive system': 1, 'The chemistry of food': 2, 'Catalysts and enzymes': 3, 'Factors affecting enzyme action': 4, 'How the digestive system works': 5, 'Making digestion efficient': 6}

Organising Animals and Plants
{'The blood': 0, 'The blood vessels': 1, 'The heart': 2, 'Helping the heart': 3, 'Breathing and gas exchange': 4, 'Tissues and organs in plants': 5, 'Transport systems in plants': 6, 'Evaporation and transpiration': 7, 'Factors affecting transpiration': 8}

Communicable diseases
{'Health and disease':

### Configure a RoBERTa model to be used to train our dataset

In [0]:
# Create RoBERTa model method for multiple instances
def create_roberta_model(labels):
  config = RobertaConfig.from_pretrained('roberta-base')
  config.num_labels = len(list(labels.values()))
  model = RobertaForSequenceClassification(config)
  return model

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [0]:
# config = RobertaConfig.from_pretrained('roberta-base')
# config.num_labels = len(list(label_to_ix.values()))
# config.num_labels = len(list(section_labels["Cell Division"].values()))

# model = RobertaForSequenceClassification(config)
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# print(list(section_labels["Cell Division"].values()))

In [59]:
# For Main-Category classification:
main_cat_model = create_roberta_model(main_category_labels)

# For Section classification:
section_model = create_roberta_model(section_labels)

# For Sub section classifcations:
subsection_models = {}
for section, subsection_labels in subsection_labels.items():
  print("Creating RoBERTa for: ", section)
  subsection_models[section] = create_roberta_model(subsection_labels)

Creating RoBERTa for:  Cell Strucuture and transport
Creating RoBERTa for:  Cell Division
Creating RoBERTa for:  Organisation and the digestive system
Creating RoBERTa for:  Organising Animals and Plants
Creating RoBERTa for:  Communicable diseases
Creating RoBERTa for:  Preventing and treating disease
Creating RoBERTa for:  Non-Communicable diseases
Creating RoBERTa for:  Photosynthesis
Creating RoBERTa for:  Respiration
Creating RoBERTa for:  The human nervous system
Creating RoBERTa for:  Reproduction
Creating RoBERTa for:  Genetics and evolution


### The following set of functions are helper-functions to create RoBERTa inputs from (question, label) pairs

In [0]:
def feature_preparation(seq, max_length = 300, zero_pad = False, include_CLS_token = True, include_SEP_token = True):
    ## Tokenzine Input
    tokens_a = tokenizer.tokenize(seq)

    ## Truncate
    if len(tokens_a) > max_length - 2:
        tokens_a = tokens_a[0:(max_length - 2)]

    ## Initialize Tokens
    tokens = []
    if include_CLS_token:
        tokens.append(tokenizer.cls_token)

    ## Add Tokens and separators
    for token in tokens_a:
        tokens.append(token)

    if include_SEP_token:
        tokens.append(tokenizer.sep_token)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    ## Input Mask 
    input_mask = [1] * len(input_ids)

    ## Zero-pad sequence lenght
    if zero_pad:
        while len(input_ids) < max_length:
            input_ids.append(0)
            input_mask.append(0)
    return torch.tensor(input_ids).unsqueeze(0), input_mask

In [69]:
# Check question preparation if correct for each dataset:
cat_question = category_dataset['question'][50]
print("Main Category")
print(cat_question)
print(feature_preparation(cat_question))
print("")

sec_question = section_dataset['question'][23]
print("Section")
print(sec_question)
print(feature_preparation(sec_question))
print("")

subsec_question = subsection_datasets["Cell Division"]['question'][9]
print("Subsection: Cell Division")
print(subsec_question)
print(feature_preparation(subsec_question))
print("")

Main Category
How do plants make their food?
(tensor([[   0, 6179,  109, 3451,  146,   49,  689,  116,    2]]), [1, 1, 1, 1, 1, 1, 1, 1, 1])

Section
What is the resolving power of an electron microscope?
(tensor([[    0,  2264,    16,     5, 20006,   476,     9,    41, 35235, 32518,
           116,     2]]), [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Subsection: Cell Division
How common is measles in the UK?
(tensor([[    0,  6179,  1537,    16, 26143,    11,     5,   987,   116,     2]]), [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])



In [0]:
class prepare_input(Dataset):
    def __init__(self, dataframe, mode="", section_to_prepare=""):
        self.len = len(dataframe)
        self.data = dataframe
        self.mode = mode
        self.section_to_prepare = section_to_prepare
        
    def __getitem__(self, index):
        question = self.data.question[index]
        label = self.data.label[index]
        X, _  = feature_preparation(question)
        if(self.section_to_prepare and self.mode == "SUBSECTIONS"):
          y = subsection_labels[self.section_to_prepare][self.data.label[index]]
        elif(self.mode == "SECTIONS"):
          y = section_labels[self.data.label[index]]
        else:
          y = main_category_labels[self.data.label[index]]
        return X, y
    
    def __len__(self):
        return self.len

### Create train-test split (80:20)

In [0]:
train_size = 0.80

In [79]:
# Main Category train-test split
category_train_dataset = category_dataset.sample(frac=train_size, random_state=200).reset_index(drop=True)
category_test_dataset = category_dataset.drop(category_train_dataset.index).reset_index(drop=True)

print("FULL Dataset: {}".format(dataset.shape))
print("TRAIN Dataset: {}".format(category_train_dataset.shape))
print("TEST Dataset: {}".format(category_test_dataset.shape))

FULL Dataset: (133, 2)
TRAIN Dataset: (989, 2)
TEST Dataset: (247, 2)


In [81]:
# Sections train-test split
section_train_dataset = section_dataset.sample(frac=train_size, random_state=200).reset_index(drop=True)
section_test_dataset = section_dataset.drop(section_train_dataset.index).reset_index(drop=True)

print("FULL Dataset: {}".format(dataset.shape))
print("TRAIN Dataset: {}".format(section_train_dataset.shape))
print("TEST Dataset: {}".format(section_test_dataset.shape))

FULL Dataset: (133, 2)
TRAIN Dataset: (989, 2)
TEST Dataset: (247, 2)


In [83]:
# For Sub-sections:
subsection_datasets_train = {}
subsection_datasets_test = {}

for section, df in subsection_datasets.items():
  train_dataset = df.sample(frac=train_size, random_state=200).reset_index(drop=True)
  test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
  subsection_datasets_train[section] = train_dataset
  subsection_datasets_test[section] = test_dataset

  print("Section name: ", section)
  print("FULL Dataset: {}".format(dataset.shape))
  print("TRAIN Dataset: {}".format(subsection_datasets_train[section].shape))
  print("TEST Dataset: {}".format(subsection_datasets_test[section].shape))

Section name:  Cell Strucuture and transport
FULL Dataset: (133, 2)
TRAIN Dataset: (244, 2)
TEST Dataset: (61, 2)
Section name:  Cell Division
FULL Dataset: (133, 2)
TRAIN Dataset: (59, 2)
TEST Dataset: (15, 2)
Section name:  Organisation and the digestive system
FULL Dataset: (133, 2)
TRAIN Dataset: (79, 2)
TEST Dataset: (20, 2)
Section name:  Organising Animals and Plants
FULL Dataset: (133, 2)
TRAIN Dataset: (102, 2)
TEST Dataset: (25, 2)
Section name:  Communicable diseases
FULL Dataset: (133, 2)
TRAIN Dataset: (102, 2)
TEST Dataset: (25, 2)
Section name:  Preventing and treating disease
FULL Dataset: (133, 2)
TRAIN Dataset: (55, 2)
TEST Dataset: (14, 2)
Section name:  Non-Communicable diseases
FULL Dataset: (133, 2)
TRAIN Dataset: (61, 2)
TEST Dataset: (15, 2)
Section name:  Photosynthesis
FULL Dataset: (133, 2)
TRAIN Dataset: (22, 2)
TEST Dataset: (6, 2)
Section name:  Respiration
FULL Dataset: (133, 2)
TRAIN Dataset: (34, 2)
TEST Dataset: (8, 2)
Section name:  The human nervous 

In [0]:
# Main Category tokenised
category_train_set = prepare_input(category_train_dataset)
category_test_set = prepare_input(category_test_dataset)

In [0]:
# Sections tokenised
section_train_set = prepare_input(section_train_dataset, "SECTIONS")
section_test_set = prepare_input(section_test_dataset, "SECTIONS")

In [0]:
# Subsections tokenised
subsections_train_set = {}
subsections_test_set = {}

for section, df in subsection_datasets_train.items():
  subsections_train_set[section] = prepare_input(df, "SUBSECTIONS", section)

for section, df in subsection_datasets_test.items():
  subsections_test_set[section] = prepare_input(df, "SUBSECTIONS", section)

### If CUDA's available, config model to use it.

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
main_cat_model = main_cat_model.cuda()
section_model = section_model.cuda()
for model in subsection_models.values():
  model = model.cuda()

In [105]:
#Check if cuda is used for data: 
print("Main Categories:")
print(category_train_set.__getitem__(0)[0].cuda())
print("")

print("Sections:")
print(section_train_set.__getitem__(0)[0].cuda())
print("")

print("Sub-Sections (Cell Division):")
print(subsections_train_set["Cell Division"].__getitem__(0)[0].cuda())

Main Categories:
tensor([[    0,   653,   109,     5, 14819,    11,  5708,   109,     7,  1026,
            10,  8276, 17487,     2]], device='cuda:0')

Sections:
tensor([[    0,   653,   109,     5, 14819,    11,  5708,   109,     7,  1026,
            10,  8276, 17487,     2]], device='cuda:0')

Sub-Sections (Cell Division):
tensor([[    0,  2264,    16,   434,  6533,     7,    28,    11,  1110,     9,
          6286, 13310,   116,     2]], device='cuda:0')


In [0]:
# Config Parameters for training the RoBERTa models
params = {'batch_size': 1,
          'shuffle': True,
          'drop_last': False,
          'num_workers': 1}

# Create the data loaders for each dataset

# CATEGORIES:
category_training_loader = DataLoader(category_train_set, **params)
category_testing_loader = DataLoader(category_test_set, **params)

# SECTIONS:
section_training_loader = DataLoader(category_train_set, **params)
section_testing_loader = DataLoader(category_test_set, **params)

# SUBSECTIONS:
subsection_training_loaders = {}
for section in subsections_train_set.keys():
  training_loader = DataLoader(subsections_train_set[section], **params)
  subsection_training_loaders[section] = training_loader
subsection_testing_loaders = {}
for section in subsections_test_set.keys():
  test_loader = DataLoader(subsections_test_set[section], **params)
  subsection_testing_loaders[section] = test_loader

## Dictionaries for the models, training & test loaders

In [0]:
model_dict = {"Main" : main_cat_model,
              "Sections" : section_model,
              "Subsections" : subsection_models}

In [0]:
training_loaders = {"Main" : category_training_loader,
                    "Sections" : section_training_loader,
                    "Subsections" : subsection_training_loaders }

testing_loaders = {"Main" : category_testing_loader,
                    "Sections" : section_testing_loader,
                    "Subsections" : subsection_testing_loaders }

In [123]:
## Test a single forward pass 
## for each training set (Category, Section, Sub-section)

# Category
cat_inp  = category_train_set.__getitem__(0)[0].cuda()
cat_output = model_dict["Main"].forward(cat_inp)
print(cat_output)
print(torch.max(cat_output[0], 1))

# Section
sec_inp  = section_train_set .__getitem__(0)[0].cuda()
sec_output = model_dict["Sections"].forward(sec_inp)
print(sec_output)
print(torch.max(sec_output[0], 1))

# Sub-Section (Cell Division)
for section, model in model_dict["Subsections"].items():
  ssec_inp = subsections_train_set[section].__getitem__(0)[0].cuda()
  ssec_output = model.forward(ssec_inp)
  print(ssec_output)
  print(torch.max(ssec_output[0], 1))

(tensor([[ 0.2398, -0.3766, -0.6867,  0.2394]], device='cuda:0',
       grad_fn=<AddmmBackward>),)
torch.return_types.max(
values=tensor([0.2398], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([0], device='cuda:0'))
(tensor([[-0.2781,  0.1012, -0.2657, -0.1549, -0.5245,  0.0255, -0.2206, -0.2533,
          0.0245, -0.2368, -0.4766, -0.2342]], device='cuda:0',
       grad_fn=<AddmmBackward>),)
torch.return_types.max(
values=tensor([0.1012], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([1], device='cuda:0'))
(tensor([[ 0.2238,  0.4885, -0.1542,  0.1852, -0.0261,  0.3045, -0.1397,  0.3980,
         -0.2841, -0.4280]], device='cuda:0', grad_fn=<AddmmBackward>),)
torch.return_types.max(
values=tensor([0.4885], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([1], device='cuda:0'))
(tensor([[-0.2910, -0.0377, -0.2043, -0.2983]], device='cuda:0',
       grad_fn=<AddmmBackward>),)
torch.return_types.max(
values=tensor([-0.0377], device='cuda:0', grad_fn=<MaxBackw

In [0]:
def train_on(model, training_loader, testing_loader, optimizer, loss_function=nn.CrossEntropyLoss()):
  max_epochs = 3

  print_at = 100 if (len(training_loader.dataset) > 300) else 50

  model = model.train()

  for epoch in tqdm_notebook(range(max_epochs)):
      print("EPOCH -- {}".format(epoch))
      for i, (question, label) in enumerate(training_loader):
          optimizer.zero_grad()
          question = question.squeeze(0)
          if torch.cuda.is_available():
            question = question.cuda()
            label = label.cuda()
          output = model.forward(question)[0]
          _, predicted = torch.max(output, 1)
          
          loss = loss_function(output, label)
          loss.backward()
          optimizer.step()
          
          if i % print_at == 0:
              correct = 0
              total = 0
              for question, label in testing_loader:
                  question = question.squeeze(0)
                  if torch.cuda.is_available():
                    question = question.cuda()
                    label = label.cuda()
                  output = model.forward(question)[0]
                  _, predicted = torch.max(output.data, 1)
                  total += label.size(0)
                  correct += (predicted.cpu() == label.cpu()).sum()
              accuracy = 100.00 * correct.numpy() / total
              print('Iteration: {}. Loss: {}. Accuracy: {}%'.format(i, loss.item(), accuracy))

In [129]:
# Train Main Category Model
# loss_function = nn.CrossEntropyLoss()
learning_rate = 1e-05
category_model_optimizer = optim.Adam(params =  model_dict["Main"].parameters(), lr=learning_rate)
train_on(model_dict["Main"], training_loaders["Main"], testing_loaders["Main"], category_model_optimizer)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


EPOCH -- 0
Iteration: 0. Loss: 1.3274734020233154. Accuracy: 0.0%
Iteration: 100. Loss: 0.6335474848747253. Accuracy: 0.0%
Iteration: 200. Loss: 0.566177487373352. Accuracy: 0.0%
Iteration: 300. Loss: 0.6211508512496948. Accuracy: 0.0%
Iteration: 400. Loss: 2.4668359756469727. Accuracy: 0.0%
Iteration: 500. Loss: 1.1589210033416748. Accuracy: 83.40080971659918%
Iteration: 600. Loss: 1.2846260070800781. Accuracy: 64.77732793522267%
Iteration: 700. Loss: 2.023427724838257. Accuracy: 42.51012145748988%
Iteration: 800. Loss: 0.22478675842285156. Accuracy: 68.01619433198381%
Iteration: 900. Loss: 0.08810043334960938. Accuracy: 80.56680161943319%
EPOCH -- 1
Iteration: 0. Loss: 0.774721086025238. Accuracy: 79.75708502024291%
Iteration: 100. Loss: 2.109522819519043. Accuracy: 73.27935222672065%
Iteration: 200. Loss: 1.5995457172393799. Accuracy: 63.56275303643725%
Iteration: 300. Loss: 0.39874768257141113. Accuracy: 75.7085020242915%
Iteration: 400. Loss: 1.0685397386550903. Accuracy: 70.85020

In [0]:
 def get_category_from(question):
  model_dict["Main"].eval()
  input_q, _ = feature_preparation(question)
  if torch.cuda.is_available():
    input_q = input_q.cuda()
  output = model_dict["Main"](input_q)[0]
  _, pred_label = torch.max(output.data, 1)
  prediction=list(main_category_labels.keys())[pred_label]
  return prediction

In [137]:
for i in range(10):
  print(category_dataset["question"][i])
  print("Actual Category label:", category_dataset.label[i])
  print("Predicted Category label: ", get_category_from(category_dataset.question[i]))
  print("")

Name one advantage and one disadvantage of a light microscope ? 
Actual Category label: Cells and Organisms
Predicted Category label:  Disease and bioenergetics

Name one advantage and one disadvantage of an electron microscope ? 
Actual Category label: Cells and Organisms
Predicted Category label:  Disease and bioenergetics

What do microscopes do? 
Actual Category label: Cells and Organisms
Predicted Category label:  Cells and Organisms

When was the microscope invented? 
Actual Category label: Cells and Organisms
Predicted Category label:  Cells and Organisms

Where are microscopes used? 
Actual Category label: Cells and Organisms
Predicted Category label:  Cells and Organisms

What is the difference between a light microscope and an electron microscope?
Actual Category label: Cells and Organisms
Predicted Category label:  Cells and Organisms

When were the first light microscopes developed?
Actual Category label: Cells and Organisms
Predicted Category label:  Cells and Organisms

W

In [138]:
# Train Section-Classifier Model
# loss_function = nn.CrossEntropyLoss()
learning_rate = 1e-05
section_model_optimizer = optim.Adam(params =  model_dict["Sections"].parameters(), lr=learning_rate)
train_on(model_dict["Sections"], training_loaders["Sections"], testing_loaders["Sections"], section_model_optimizer)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


EPOCH -- 0
Iteration: 0. Loss: 2.4992432594299316. Accuracy: 0.0%
Iteration: 100. Loss: 0.5006554126739502. Accuracy: 0.0%
Iteration: 200. Loss: 0.7347457408905029. Accuracy: 0.0%
Iteration: 300. Loss: 1.6232085227966309. Accuracy: 0.0%
Iteration: 400. Loss: 0.9968206882476807. Accuracy: 0.4048582995951417%
Iteration: 500. Loss: 1.9106199741363525. Accuracy: 0.0%
Iteration: 600. Loss: 1.2735066413879395. Accuracy: 56.68016194331984%
Iteration: 700. Loss: 0.4117436408996582. Accuracy: 36.8421052631579%
Iteration: 800. Loss: 0.8885021209716797. Accuracy: 77.32793522267207%
Iteration: 900. Loss: 1.4557228088378906. Accuracy: 84.21052631578948%
EPOCH -- 1
Iteration: 0. Loss: 0.41133832931518555. Accuracy: 82.18623481781377%
Iteration: 100. Loss: 1.0830409526824951. Accuracy: 73.27935222672065%
Iteration: 200. Loss: 0.28741884231567383. Accuracy: 78.94736842105263%
Iteration: 300. Loss: 0.11815404891967773. Accuracy: 76.51821862348179%
Iteration: 400. Loss: 0.48943662643432617. Accuracy: 84

In [0]:
 def get_section_from(question):
  model_dict["Sections"].eval()
  input_q, _ = feature_preparation(question)
  if torch.cuda.is_available():
    input_q = input_q.cuda()
  output = model_dict["Sections"](input_q)[0]
  _, pred_label = torch.max(output.data, 1)
  prediction=list(section_labels.keys())[pred_label]
  return prediction

In [143]:
for i in range(10):
  print(section_dataset["question"][i])
  print("Actual Category label:", section_dataset.label[i])
  print("Predicted Category label: ", get_section_from(section_dataset.question[i]))
  print("")

Name one advantage and one disadvantage of a light microscope ? 
Actual Category label: Cell Strucuture and transport
Predicted Category label:  Cell Division

Name one advantage and one disadvantage of an electron microscope ? 
Actual Category label: Cell Strucuture and transport
Predicted Category label:  Cell Division

What do microscopes do? 
Actual Category label: Cell Strucuture and transport
Predicted Category label:  Cell Strucuture and transport

When was the microscope invented? 
Actual Category label: Cell Strucuture and transport
Predicted Category label:  Cell Strucuture and transport

Where are microscopes used? 
Actual Category label: Cell Strucuture and transport
Predicted Category label:  Cell Strucuture and transport

What is the difference between a light microscope and an electron microscope?
Actual Category label: Cell Strucuture and transport
Predicted Category label:  Cell Strucuture and transport

When were the first light microscopes developed?
Actual Category 

In [151]:
# Train Subsection-Classifier Model
# loss_function = nn.CrossEntropyLoss()
learning_rate = 1e-05

for section, model in model_dict["Subsections"].items():
  print("BEGIN TRAINING FOR SECTION: {} ------------------------------ ".format(section))
  print("")
  subsection_model_optimizer = optim.Adam(params =  model.parameters(), lr=learning_rate)
  train_on(model, training_loaders["Subsections"][section], testing_loaders["Subsections"][section], subsection_model_optimizer)
  print("END OF TRAINING FOR SECTION: {} ------------------------------ ".format(section))
  print("")

BEGIN TRAINING FOR SECTION: Cell Strucuture and transport ------------------------------ 



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


EPOCH -- 0



RuntimeError: ignored

In [0]:
 def get_subsection_from(question, section):
  model.eval()
  input_q, _ = feature_preparation(question)
  if torch.cuda.is_available():
    input_q = input_q.cuda()
  output = model(input_q)[0]
  _, pred_label = torch.max(output.data, 1)
  # prediction=list(label_to_ix.keys())[pred_label]
  prediction=list(subsection_labels[section].keys())[pred_label]
  return prediction

In [51]:
section_labels["Cell Division"].keys()

dict_keys(['Cell division', 'Growth and differentiation', 'Stem cells', 'Stem cell dilemmas'])

In [56]:
# print(section_dfs_train["Cell Division"].question[50])
for i in range(10):
  print(subsection_datasets_train["Cell Division"].question[i])
  print("Actual subsection label:", subsection_datasets_train["Cell Division"].label[i])
  print("Predicted label: ", get_subsection_from(subsection_datasets_train["Cell Division"].question[i], "Cell Division"))
  print("")

What is growth defined to be in terms of mitosis?
Actual subsection label: Growth and differentiation
Predicted label:  Growth and differentiation

Give three sources of stem cells? 
Actual subsection label: Stem cells
Predicted label:  Stem cells

Why are measle numbers low in the UK?
Actual subsection label: Cell division
Predicted label:  Growth and differentiation

How is the embryo formed?
Actual subsection label: Stem cells
Predicted label:  Growth and differentiation

How can people have the best chance of long-term survival with HIV/AIDS?
Actual subsection label: Cell division
Predicted label:  Growth and differentiation

What are the main symptoms of measles?
Actual subsection label: Cell division
Predicted label:  Growth and differentiation

What can stem cells from plant meristems be used for?
Actual subsection label: Stem cells
Predicted label:  Growth and differentiation

Identify the differences between a stem cell and a normal body cell? 
Actual subsection label: Stem ce

# LAYER 2: Untouched as of yet.

In [0]:
from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample

from transformers.data.metrics.squad_metrics import compute_predictions_logits

model_name_or_path = "ktrapeznikov/albert-xlarge-v2-squad-v2"
# model_name_or_path = "bert-base-uncased"

output_dir = ""


def to_list(tensor):
    return tensor.detach().cpu().tolist()
    
# Config
n_best_size = 1
max_answer_length = 30
do_lower_case = True
null_score_diff_threshold = 0.0

# Setup model
config_class, model_class, tokenizer_class = (
    AlbertConfig, AlbertForQuestionAnswering, AlbertTokenizer)


config = config_class.from_pretrained(model_name_or_path)

tokenizer = tokenizer_class.from_pretrained(model_name_or_path, do_lower_case=True)

model = model_class.from_pretrained(model_name_or_path, config=config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

processor = SquadV2Processor()

def run_prediction(question_texts, context_text):
    """Setup function to compute predictions"""
    examples = []

    for i, question_text in enumerate(question_texts):
        example = SquadExample(
            qas_id=str(i),
            question_text=question_text,
            context_text=context_text,
            answer_text=None,
            start_position_character=None,
            title="Predict",
            is_impossible=False,
            answers=None,
        )

        examples.append(example)

    features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=384,
        doc_stride=128,
        max_query_length=64,
        is_training=False,
        return_dataset="pt",
        threads=1,
    )

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

    all_results = []

    for batch in eval_dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs)

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)

                output = [to_list(output[i]) for output in outputs]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
                all_results.append(result)

    output_prediction_file = "predictions.json"
    output_nbest_file = "nbest_predictions.json"
    output_null_log_odds_file = "null_predictions.json"

    predictions = compute_predictions_logits(
        examples,
        features,
        all_results,
        n_best_size,
        max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,  # verbose_logging
        True,  # version_2_with_negative
        null_score_diff_threshold,
        tokenizer,
    )

    return predictions

In [0]:
context = "Animals need to eat food to get their energy. But green plants and algae do not. Instead they make their own food in a process called photosynthesis. Almost all life on Earth depends upon this process. Photosynthesis is also important in maintaining the levels of oxygen and carbon dioxide in the atmosphere. These are the things that plants need for photosynthesis: carbon dioxide, water, light (a source of energy) These are the things that plants make by photosynthesis: glucose and oxygen Photosynthesis takes place inside plant cells in small objects called chloroplasts. Chloroplasts contain a green substance called chlorophyll. This absorbs the light energy needed to make photosynthesis happen. Plants and algae can only carry out photosynthesis in the light. Plants get carbon dioxide from the air through their leaves, and water from the ground through their roots. Light energy comes from the Sun. The oxygen produced is released into the air from the leaves. The glucose produced can be turned into other substances, such as starch and plant oils, which are used as an energy store. This energy can be released by respiration."
questions = ["What are the products of photosynthesis?", 
             "What colour is chlorophyll?",
             "Where does photosynthesis take place?",
             "When do plants photosynthesise?",
             "What are the reactants of photosynthesis?"]

# Run method
predictions = run_prediction(questions, context)

# Print results
for key in predictions.keys():
  print(predictions[key])

add example index and unique id: 100%|██████████| 5/5 [00:00<00:00, 13374.69it/s]


glucose and oxygen
green
inside plant cells
in the light
glucose and oxygen
